In [23]:
video_folder = "AlgonautsVideos268_All_30fpsmax"

In [24]:
import os
import cv2
import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model

In [25]:
# Initialize ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet')
res_model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)


In [32]:
def preprocess_video(video_path, frame_rate=15):
    # Capture video
    vidcap = cv2.VideoCapture(video_path)
    success, frame = vidcap.read()
    count = 0
    frames = []

    while success:
        if count % (30 // frame_rate) == 0:  # Reduce frame rate
            # Resize frame to match the input size expected by ResNet50
            resized_frame = cv2.resize(frame, (224, 224))  # Resize to 224x224

            # Preprocess frame
            img = image.img_to_array(resized_frame)  # Use resized frame
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)

            # Extract features
            features = res_model.predict(img)

            # Append features
            frames.append(features)
        
        success, frame = vidcap.read()
        count += 1

    # Stack for 3D input
    video_tensor = np.stack(frames, axis=0)
    return video_tensor


In [33]:
# Example usage
video_tensor = preprocess_video('AlgonautsVideos268_All_30fpsmax/0002_0-0-4-3146384004.mp4')

1/1 [==============================] - 0s 87ms/step


In [34]:
video_tensor.shape

(46, 1, 2048)

In [35]:
video_tensor

array([[[0.22161636, 0.7109571 , 0.27197546, ..., 2.5333972 ,
         0.5847285 , 1.0432433 ]],

       [[0.29266238, 0.69181544, 0.21708311, ..., 2.6417542 ,
         0.5042521 , 0.85307056]],

       [[0.3136437 , 0.41500956, 0.3766173 , ..., 2.8379903 ,
         0.36808145, 0.34662256]],

       ...,

       [[0.14063771, 0.8883256 , 0.57336164, ..., 4.1228256 ,
         0.17979139, 0.8579453 ]],

       [[0.13395272, 0.88448614, 0.32257167, ..., 3.6319773 ,
         0.16747952, 0.88167423]],

       [[0.22048873, 0.6794995 , 0.37961558, ..., 4.004434  ,
         0.1377652 , 1.2170678 ]]], dtype=float32)

In [36]:
def process_and_save_videos(input_folder, output_folder, max_files=10):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process and save first 'max_files' MP4 files
    processed_count = 0
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4") and processed_count < max_files:
            video_path = os.path.join(input_folder, filename)
            processed_data = preprocess_video(video_path)

            # Save processed data
            output_path = os.path.join(output_folder, f"processed_{filename}.pkl")
            with open(output_path, 'wb') as file:
                pickle.dump(processed_data, file)

            processed_count += 1


In [37]:
input_folder = video_folder  # Update with your input folder path
output_folder = 'preprocessed_videos'  # Update with your output folder path
process_and_save_videos(input_folder, output_folder)

1/1 [==============================] - 0s 120ms/step


In [38]:
from sklearn.decomposition import PCA

def dimension_reduction_pca(video_tensor, n_components=0.95):
    """
    Applies PCA to reduce the dimensions of the video tensor.

    Parameters:
    video_tensor (numpy.ndarray): The video tensor to be reduced in dimension.
    n_components (float or int): Number of components to keep. 
                                 If 0 < n_components < 1, select the number of components such that 
                                 the cumulative explained variance ratio is greater than n_components.

    Returns:
    numpy.ndarray: The transformed video tensor with reduced dimensions.
    """

    # Reshape the video tensor for PCA
    # Assuming the shape of the video_tensor is (num_frames, 1, num_features)
    # We need to reshape it to (num_frames, num_features)
    reshaped_tensor = video_tensor.reshape(video_tensor.shape[0], -1)

    # Initialize PCA
    pca = PCA(n_components=n_components)

    # Apply PCA
    reduced_tensor = pca.fit_transform(reshaped_tensor)

    return reduced_tensor

In [39]:
pca_video_tensor = dimension_reduction_pca(video_tensor)

In [40]:
pca_video_tensor

array([[ 6.2146063 ,  6.8615575 , -6.030319  , ..., -0.20338462,
        -1.0027943 ,  0.28454036],
       [ 6.951216  ,  7.2890635 , -6.0229692 , ..., -0.5103349 ,
        -0.6265724 ,  0.3786705 ],
       [ 7.517687  ,  7.208666  , -6.953383  , ..., -0.8728668 ,
         0.40504184, -0.9677295 ],
       ...,
       [-8.936657  , 12.265885  ,  2.9073694 , ..., -1.2729971 ,
        -0.9938903 ,  1.8501117 ],
       [-8.781005  , 12.595076  ,  3.1718147 , ..., -1.1237453 ,
         1.8169147 , -0.37966007],
       [-9.121401  , 11.6159935 ,  3.8334196 , ...,  1.047599  ,
         0.19776364, -0.58459234]], dtype=float32)

In [47]:
# Define the function to apply PCA and save reduced videos
def save_reduced_videos(source_folder, target_folder, n_components=0.95):

    # Process each tensor in the source folder
    for file_name in os.listdir(source_folder):
        file_path = os.path.join(source_folder, file_name)
        if file_path.endswith('.pkl'):
            # Load the video tensor
            with open(file_path, 'rb') as file:
                video_tensor = pickle.load(file)

            # Reduce dimensionality
            reduced_tensor = dimension_reduction_pca(video_tensor, n_components)

            # Save the reduced data
            reduced_file_path = os.path.join(target_folder, file_name.replace('.pkl', '_reduced.pkl'))
            with open(reduced_file_path, 'wb') as reduced_file:
                pickle.dump(reduced_tensor, reduced_file)
                

In [48]:
source_folder = 'preprocessed_videos'
target_folder = 'reduced_videos'
save_reduced_videos(source_folder, target_folder)

reduced
reduced
reduced
reduced
reduced
reduced
reduced
reduced
reduced
reduced
